In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

In [17]:
train = pd.read_json('train.json')

train['ingredients_str'] = train['ingredients'].apply(lambda x: ' '.join(x))

le = LabelEncoder()
train['cuisine'] = le.fit_transform(train['cuisine'])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(train['ingredients_str'], train['cuisine'], test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=20000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 100
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [19]:
embeddings_index = {}
with open('glove.6B.100d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        except ValueError:
            print(f"Skipping line with formatting issue: {line[:50]}...")

embedding_dim = 100
word_index = tokenizer.word_index
num_words = min(20000, len(word_index) + 1)

embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= 20000:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


Skipping line with formatting issue: standard -0.13416 0.47697 0.45242 0.2767 -0.25912 ...


In [24]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

model = Sequential()
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_dim,
                    input_length=max_len,
                    weights=[embedding_matrix],
                    trainable=False))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(le.classes_), activation='softmax'))

optimizer = Adam(learning_rate=0.00001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [26]:
history = model.fit(X_train_padded, y_train, epochs=50, validation_data=(X_test_padded, y_test), batch_size=64)

model.save('improvedv2_cuisine_classification_model.h5')

loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Epoch 1/50
498/498 [==============================] - 13s 26ms/step - loss: 2.8054 - accuracy: 0.1838 - val_loss: 2.6554 - val_accuracy: 0.2106
Epoch 2/50
498/498 [==============================] - 12s 24ms/step - loss: 2.6423 - accuracy: 0.1976 - val_loss: 2.5408 - val_accuracy: 0.2249
Epoch 3/50
498/498 [==============================] - 11s 23ms/step - loss: 2.5460 - accuracy: 0.2220 - val_loss: 2.4701 - val_accuracy: 0.2415
Epoch 4/50
498/498 [==============================] - 11s 23ms/step - loss: 2.4918 - accuracy: 0.2378 - val_loss: 2.4241 - val_accuracy: 0.2591
Epoch 5/50
498/498 [==============================] - 12s 25ms/step - loss: 2.4481 - accuracy: 0.2532 - val_loss: 2.3678 - val_accuracy: 0.2867
Epoch 6/50
498/498 [==============================] - 11s 22ms/step - loss: 2.3974 - accuracy: 0.2792 - val_loss: 2.3162 - val_accuracy: 0.3030
Epoch 7/50
498/498 [==============================] - 11s 23ms/step - loss: 2.3557 - accuracy: 0.2946 - val_loss: 2.2785 - val_accuracy:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


249/249 [==============================] - 3s 11ms/step - loss: 1.8285 - accuracy: 0.4562
Model Accuracy: 45.62%
